## 0.Data 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 1.0 Data Loading

In [4]:
meta=pd.read_json(r"C:\Users\nisha\Downloads\meta_All_Beauty.jsonl.gz",lines=True)

In [6]:
rev=pd.read_json(r"C:\Users\nisha\Downloads\All_Beauty.jsonl\All_Beauty.jsonl",lines=True)

In [8]:
rev.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,True
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,True
2,5,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,True
3,1,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2022-01-28 18:13:50.220,0,True
4,5,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-30 10:02:43.534,0,True


In [10]:
meta.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,NaN
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,NaN
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,NaN
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,NaN
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,NaN


## 2.0 Basic Checks

In [18]:
print(meta.columns)
print(rev.columns)


Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together'],
      dtype='object')
Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')


In [20]:
meta.shape

(112590, 14)

In [22]:
meta.isnull().sum()

main_category           0
title                   0
average_rating          0
rating_number           0
features                0
description             0
price               94886
images                  0
videos                  0
store               11331
categories              0
details                 0
parent_asin             0
bought_together    112590
dtype: int64

In [23]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112590 entries, 0 to 112589
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   main_category    112590 non-null  object 
 1   title            112590 non-null  object 
 2   average_rating   112590 non-null  float64
 3   rating_number    112590 non-null  int64  
 4   features         112590 non-null  object 
 5   description      112590 non-null  object 
 6   price            17704 non-null   float64
 7   images           112590 non-null  object 
 8   videos           112590 non-null  object 
 9   store            101259 non-null  object 
 10  categories       112590 non-null  object 
 11  details          112590 non-null  object 
 12  parent_asin      112590 non-null  object 
 13  bought_together  0 non-null       float64
dtypes: float64(3), int64(1), object(10)
memory usage: 12.0+ MB


In [24]:
meta['price'].nunique()

3531

In [25]:
rev.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701528 entries, 0 to 701527
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   rating             701528 non-null  int64         
 1   title              701528 non-null  object        
 2   text               701528 non-null  object        
 3   images             701528 non-null  object        
 4   asin               701528 non-null  object        
 5   parent_asin        701528 non-null  object        
 6   user_id            701528 non-null  object        
 7   timestamp          701528 non-null  datetime64[ns]
 8   helpful_vote       701528 non-null  int64         
 9   verified_purchase  701528 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 48.8+ MB


In [26]:
rev.isnull().sum()

rating               0
title                0
text                 0
images               0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64

**Null Values Handiling**

In [31]:
# Example: fill first 100,000 rows with 9.99
meta.loc[:9999, "price"] = meta.loc[:9999, "price"].fillna(9.99)
meta.loc[10000:19999, "price"] = meta.loc[10000:19999, "price"].fillna(19.99)
meta.loc[20000:29999, "price"] = meta.loc[20000:29999, "price"].fillna(29.99)
meta.loc[30000:39999, "price"] = meta.loc[30000:39999, "price"].fillna(39.99)
meta.loc[40000:49999, "price"] = meta.loc[40000:49999, "price"].fillna(49.99)
meta.loc[50000:59999, "price"] = meta.loc[50000:59999, "price"].fillna(59.99)
meta.loc[60000:69999, "price"] = meta.loc[60000:69999, "price"].fillna(69.99)
meta.loc[70000:79999, "price"] = meta.loc[70000:79999, "price"].fillna(79.99)
meta.loc[80000:89999, "price"] = meta.loc[80000:89999, "price"].fillna(89.99)
meta.loc[90000:99999, "price"] = meta.loc[90000:99999, "price"].fillna(99.99)
meta.loc[100000:, "price"] = meta.loc[100000: "price"].fillna(100.99)
# Check
print(meta["price"].isna().sum())


0


In [32]:
meta['main_category'].nunique()

2

**Creating New Features**

In [38]:
from IPython.display import Image, display

In [40]:
def extract_first_image(img_list):
    try:
        if isinstance(img_list, str):
            img_list = ast.literal_eval(img_list)  # convert string to list
        if isinstance(img_list, list) and len(img_list) > 0:
            return img_list[0].get("hiRes") or img_list[0].get("large") or None
    except:
        return None
    return None

In [42]:
meta["image_url"] = meta["images"].apply(extract_first_image)

In [43]:
meta.columns

Index(['main_category', 'title', 'average_rating', 'rating_number', 'features',
       'description', 'price', 'images', 'videos', 'store', 'categories',
       'details', 'parent_asin', 'bought_together', 'image_url'],
      dtype='object')

**Droppng Unnecessary Features**

In [47]:
meta=meta.drop(['videos','store','bought_together'],axis=1)

In [49]:
meta[['title', 'average_rating', 'rating_number', 'price', 'image_url']]

,title,average_rating,rating_number,price,image_url
0,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,9.99,https://m.media-amazon.com/images/I/41qfjSfqNy...
1,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,9.99,https://m.media-amazon.com/images/I/41b+11d5ig...
2,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,9.99,https://m.media-amazon.com/images/I/31bz+uqzWC...
3,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,9.99,https://m.media-amazon.com/images/I/515iwxdKS1...
4,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,9.99,https://m.media-amazon.com/images/I/31TgqAZ8kQ...
...,...,...,...,...,...
112585,"TOPREETY 24""120gr 3/4 Full Head clip in hair e...",5.0,3,100.99,https://m.media-amazon.com/images/I/41drltfQaq...
112586,"Pets Playmate Pet Grooming Glove,Gentle Deshed...",5.0,1,100.99,https://m.media-amazon.com/images/I/41J8t6Psuf...
112587,[10Pack] Makeup Brushes Set Cosmetics Tools Ki...,4.3,5,100.99,https://m.media-amazon.com/images/I/512r4-h8xE...
112588,Xcoser Pretty Party Anna Wig Hair Tails Hair S...,4.0,9,100.99,https://m.media-amazon.com/images/I/51MsF9Qef0...


## Top Popular Products

In [52]:
from IPython.display import Image, display

def get_popular_products(n=10):
    popular = meta.sort_values("rating_number", ascending=False).head(n)
    return popular[["title", "average_rating", "rating_number", "price", "image_url"]]

def show_popular_products(n=10):
    popular = get_popular_products(n)
    print(f"🔥 Top {n} Popular Products:\n")
    for _, row in popular.iterrows():
        print(f"⭐ {row['title']} | Rating: {row['average_rating']} | Price: {row['price']} | {row['rating_number']} reviews")
        if row["image_url"]:
            display(Image(url=row["image_url"], width=250, height=150))
show_popular_products(10)

🔥 Top 10 Popular Products:

⭐ Elle 18 Eye Drama Kajal, Bold Black, 0.35 g | Rating: 4.2 | Price: 5.4 | 30206 reviews


⭐ Maybelline New York Colossal Bold Eyeliner, Black, 3g | Rating: 4.3 | Price: 8.0 | 27308 reviews


⭐ JIMIRE Fake Eyelashes Fluffy Volume 3D Faux Mink Eyelashes Natural Lashes Pack 5 Pairs | Rating: 4.4 | Price: 59.99 | 16062 reviews


⭐ THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1.01 Fl Oz (Pack of 1) | Rating: 4.5 | Price: 59.99 | 15264 reviews


⭐ THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1.01 Fl Oz (Pack of 1) | Rating: 4.5 | Price: 39.99 | 15185 reviews


⭐ Pro by Oral-B 2500 3D White Pink Toothbrush with Case | Rating: 4.7 | Price: 100.99 | 14700 reviews


⭐ EmaxDesign Makeup Brushes 17 Pieces Premium Synthetic Foundation Brush Powder Blending Blush Concealer Eye Face Liquid Powder Cream Cosmetics Brushes Kit (Rose Gold) | Rating: 4.6 | Price: 11.99 | 13676 reviews


⭐ Oral-B Pro 2900 Electric Rechargeable Toothbrush (Two Handle Pack) | Rating: 4.7 | Price: 100.99 | 12870 reviews


⭐ 3/4" Bright Red Round Color Coding Circle Dot Labels on a Roll, Semi-Gloss, 1000 Stickers.75 inch Diameter. | Rating: 4.8 | Price: 89.99 | 12205 reviews


⭐ 1 Inch Round Thank You Sticker Labels with Red Hearts, 1000 Stickers per Roll | Rating: 4.8 | Price: 89.99 | 12205 reviews


In [56]:
meta["popularity_score"] = meta["rating_number"] * meta["average_rating"]

# Select top 20
popular_products = (
    meta.sort_values("popularity_score", ascending=False)
        .head(20)[["title", "average_rating", "rating_number", "image_url"]]
)

popular_products.reset_index(drop=True, inplace=True)
popular_products.head()

,title,average_rating,rating_number,image_url
0,"Elle 18 Eye Drama Kajal, Bold Black, 0.35 g",4.2,30206,https://m.media-amazon.com/images/I/41Zqm0xBTO...
1,"Maybelline New York Colossal Bold Eyeliner, Bl...",4.3,27308,https://m.media-amazon.com/images/I/31Xq6wNJwF...
2,JIMIRE Fake Eyelashes Fluffy Volume 3D Faux Mi...,4.4,16062,https://m.media-amazon.com/images/I/5148arXWI4...
3,Pro by Oral-B 2500 3D White Pink Toothbrush wi...,4.7,14700,https://m.media-amazon.com/images/I/416LBdZPzD...
4,"THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1...",4.5,15264,https://m.media-amazon.com/images/I/21X+1j4ijY...


## Content Recommendations For Selected Popular Product

In [59]:
meta['description'] = meta['description'].fillna("")
meta['categories'] = meta['categories'].fillna("").astype(str)

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel

In [67]:
if 'description' not in meta.columns:
    meta['description'] = meta['title']

In [71]:
tfidf = TfidfVectorizer(stop_words='english', max_features=50000)
tfidf_matrix = tfidf.fit_transform(meta['title'])

In [73]:
tfidf_matrix

<112590x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 1560768 stored elements in Compressed Sparse Row format>

In [75]:
nn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
nn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [77]:
def get_content_recommendations(title, n=10):
    idx = meta.index[meta['title'] == title].tolist()
    if not idx:
        print("Product not found!")
        return
    idx = idx[0]
    
    distances, indices = nn.kneighbors(tfidf_matrix[idx], n_neighbors=n+1)
    
    recommended_indices = indices[0][1:]
    recommended = meta.iloc[recommended_indices][['title', 'description', 'image_url']]
    
    for i, row in recommended.iterrows():
        display(HTML(f"<h3>{row['title']}</h3><p>{row['description']}</p><img src='{row['image_url']}' width='200'><hr>"))

In [83]:
meta['title'].unique

<bound method Series.unique of 0         Howard LC0008 Leather Conditioner, 8-Ounce (4-...
1         Yes to Tomatoes Detoxifying Charcoal Cleanser ...
2          Eye Patch Black Adult with Tie Band (6 Per Pack)
3         Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...
4         Precision Plunger Bars for Cartridge Grips – 9...
                                ...                        
112585    TOPREETY 24"120gr 3/4 Full Head clip in hair e...
112586    Pets Playmate Pet Grooming Glove,Gentle Deshed...
112587    [10Pack] Makeup Brushes Set Cosmetics Tools Ki...
112588    Xcoser Pretty Party Anna Wig Hair Tails Hair S...
112589    DVIO Men's Voyage Perfume, Spicy woody fragran...
Name: title, Length: 112590, dtype: object>

In [85]:
get_content_recommendations("Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)", n=5)


##  Collaborative Recommendations For Selected Popular/Content Product

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
from sklearn.neighbors import NearestNeighbors

In [90]:
def get_popular_products(n=10):
    popular = meta.sort_values("rating_number", ascending=False).head(n)
    return popular[['parent_asin', 'title', 'average_rating', 'rating_number', 'image_url']]

In [94]:
user_codes = rev['user_id'].astype('category').cat.codes
item_codes = rev['asin'].astype('category').cat.codes
sparse_matrix = csr_matrix((rev['rating'], (user_codes, item_codes)))

In [110]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1)
model_knn.fit(sparse_matrix.T)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=6)

In [112]:
asin_to_idx = dict(zip(rev['asin'], item_codes))
idx_to_asin = dict(zip(item_codes, rev['asin']))

In [118]:
def get_collab_recommendations(asin, n=5):
    if asin not in asin_to_idx:
        print("ASIN not found in ratings data.")
        return
    
    item_idx = asin_to_idx[asin]
    distances, indices = model_knn.kneighbors(sparse_matrix.T[item_idx], n_neighbors=n+1)
    neighbor_asins = [idx_to_asin[i] for i in indices.flatten()[1:]]
    recommended = meta[meta['parent_asin'].isin(neighbor_asins)][
        ['parent_asin', 'title', 'average_rating', 'rating_number', 'image_url']
    ].drop_duplicates()
    print(f"\n🔹 Collaborative recommendations for {asin}:")
    for _, row in recommended.iterrows():
        display(HTML(f"""
            <div style="border:1px solid #ccc; padding:10px; margin:10px;">
                <h3>{row['title']}</h3>
                <p>ASIN: {row['parent_asin']}</p>
                <p>⭐ {row['average_rating']} ({row['rating_number']} ratings)</p>
                <img src="{row['image_url']}" width="150">
            </div>
        """))

In [134]:
def recommend_for_popular(n_popular=1, n_recs=5):
    popular_products = get_popular_products(n_popular)
    
    for _, row in popular_products.iterrows():
        print(f"\n=== Popular Product: {row['title']} ({row['parent_asin']}) ===")
        display(HTML(f"<img src='{row['image_url']}' width='150'><br>"))
        get_collab_recommendations(row['parent_asin'], n=n_recs)

In [142]:
recommend_for_popular(n_popular=4, n_recs=5)


=== Popular Product: Elle 18 Eye Drama Kajal, Bold Black, 0.35 g (B01LNT83KK) ===



🔹 Collaborative recommendations for B01LNT83KK:



=== Popular Product: Maybelline New York Colossal Bold Eyeliner, Black, 3g (B07S141T2R) ===



🔹 Collaborative recommendations for B07S141T2R:



=== Popular Product: JIMIRE Fake Eyelashes Fluffy Volume 3D Faux Mink Eyelashes Natural Lashes Pack 5 Pairs (B08CVJJ518) ===



🔹 Collaborative recommendations for B08CVJJ518:



=== Popular Product: THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1.01 Fl Oz (Pack of 1) (B01MDTVZTZ) ===



🔹 Collaborative recommendations for B01MDTVZTZ:


In [138]:
# Show top 10 popular products
popular_products = get_popular_products(10)
print("Top 10 Popular Products:")
for i, row in popular_products.iterrows():
    print(f"{i}: {row['title']} (ASIN: {row['parent_asin']})")

Top 10 Popular Products:
50834: Elle 18 Eye Drama Kajal, Bold Black, 0.35 g (ASIN: B01LNT83KK)
1403: Maybelline New York Colossal Bold Eyeliner, Black, 3g (ASIN: B07S141T2R)
51573: JIMIRE Fake Eyelashes Fluffy Volume 3D Faux Mink Eyelashes Natural Lashes Pack 5 Pairs (ASIN: B08CVJJ518)
55874: THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1.01 Fl Oz (Pack of 1) (ASIN: B01MDTVZTZ)
32240: THE ORDINARY Niacinamide 10% + Zinc 1% 30ml, 1.01 Fl Oz (Pack of 1) (ASIN: B06VSX2B1S)
112578: Pro by Oral-B 2500 3D White Pink Toothbrush with Case (ASIN: B01EY8LK4I)
65359: EmaxDesign Makeup Brushes 17 Pieces Premium Synthetic Foundation Brush Powder Blending Blush Concealer Eye Face Liquid Powder Cream Cosmetics Brushes Kit (Rose Gold) (ASIN: B07G19ZXWB)
109045: Oral-B Pro 2900 Electric Rechargeable Toothbrush (Two Handle Pack) (ASIN: B072VBF6M4)
88581: 3/4" Bright Red Round Color Coding Circle Dot Labels on a Roll, Semi-Gloss, 1000 Stickers.75 inch Diameter. (ASIN: B07VPR5W4D)
88535: 1 Inch Round Than

In [140]:
choice_index = int(input("Enter the index of the product you want recommendations for: "))
selected_product_asin = popular_products.iloc[choice_index]['parent_asin']
get_collab_recommendations(selected_product_asin, n=5)

Enter the index of the product you want recommendations for:  0



🔹 Collaborative recommendations for B01LNT83KK:
